### llm module

In [1]:
import pandas as pd
import numpy as np
import os
from openai import AzureOpenAI
from dotenv import load_dotenv

# del os.environ["API_VERSION_4o"]
load_dotenv()

# print(os.getenv("AZURE_OPENAI_4o_API_KEY"))
# print(os.getenv("API_VERSION_4o"))
# print(os.getenv("AZURE_OPENAI_4o_ENDPOINT"))

def generate_response(query):
    schema_info=""
    with open("temp/data_description.txt","r") as file:
        for reader in file.readlines():
            schema_info+=reader


    client = AzureOpenAI(
    api_key = os.getenv("AZURE_OPENAI_4o_API_KEY"),  
    api_version = os.getenv("API_VERSION_4o"),
    azure_endpoint = os.getenv("AZURE_OPENAI_4o_ENDPOINT")
    )


    response = client.chat.completions.create(
        model="gpt-4o", # model = "deployment_name".
        messages=[
            {"role": "system", "content": f"""You have access to a pandas dataframe named df. \
    Here is a name and decription of each column in the dataframe

    {schema_info}

    Don't asssume any other column and feature name, it should be exactly the same. \
    Given a user question about the dataframe,Provide only the Python code to answer it.\
    Don't assume you have access to any libraries other than built-in Python ones and pandas only. \
    You don't have any other data source other than dataframe df. \
    Make sure to refer only to the variables mentioned above. \
    Give the script to wrap it inside the custom function name generated_response(). \
    Return a single, new dataframe with only relevant and suitable column names according to the user query without modifying df \
    Give only required columns. \
    Do not add any print statements. \
    Do not alter placeholder values . \
    DO NOT REMOVE ANGULAR BRACKETS"""},

    {"role": "user", "content": f"{query}"},


        ]
    )

    # print(response.choices[0].message.content)

    # script=""
    # for line in response.choices[0].message.content.split("```")[1].split("\n")[1:]:
    #     script+=line+"\n"

    script=response.choices[0].message.content
    temp=script.split("```")[1]
    script=temp[6:]

    return script

### llm_advanced module

In [2]:
import pandas as pd
import numpy as np
import os
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv()

def generate_response_from_filtered_data(dataframe,query):

    schema_info=""
    with open("temp/data_description.txt","r") as file:
        for reader in file.readlines():
            schema_info+=reader

    client = AzureOpenAI(
    api_key = os.getenv("AZURE_OPENAI_4o_API_KEY"),  
    api_version = os.getenv("API_VERSION_4o"),
    azure_endpoint = os.getenv("AZURE_OPENAI_4o_ENDPOINT")
    )

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": f"""
             You are responsible for explaining the answer to the user based on the dataframe, schema and the query provided to you.\
    The following dataframe is already filtered according to user query and columns renamed: {dataframe.to_csv(index=False)} \
    Assume it contains all the information asked in query. \
    This is the description of original dataset: {schema_info} \
    If query cannot be answered from the provided data, simply explain the reason for the same.\
    Do not give any tabular data. \
    Do not include any reasonings and warnings, directly respond to the query. \
    Do not give any warnings. \
    Do not remove angular <> brackets, keep those values as it is. \
    Do not alter placeholder values . \
    Empty dataframe means there were no such records."""} ,
    

    {"role": "user", "content": f"{query}"},

        ]
    )

    RESPONSE=response.choices[0].message.content

    return RESPONSE

### anonymizer module

In [3]:
# from langchain_experimental.data_anonymizer import PresidioReversibleAnonymizer
# from presidio_analyzer import PatternRecognizer, Pattern
# import pandas as pd

# # For dataframe only
# columns_to_anonymize = ["customer name","IC01 Code","sysname","TMKey","sdwan_ip_admin"] # list all the column names to anonymize
# columns_to_anonymize_deny_list = ["customer name"] # list all the column names to anonymize via deny list

# # Read data as dataset for using anonimization via deny list
# dataset = pd.read_excel("Power BI data-Security.xlsx",skiprows=2)

# anonymizer = PresidioReversibleAnonymizer(
#     analyzed_fields=[], # choose inbuilt fields to be considered for anonimyzation
#     faker_seed=42,
# )

# # Define custom regex patterns to add in presidio for anonimyzation --> anonymized entries will be named 'support_entity_{occurence_no}'
# SYSTEM_NAME_pattern = Pattern(name="sysname_pattern", regex="[a-z]{4}[0-9]{3,4}", score=0.8)
# IC01_pattern = Pattern(name="ic01_pattern", regex="[0-9]{4,7}",score=1.0)
# IPV4_IP_pattern = Pattern("customer_name_alnum_pattern",regex="[0-9]{1,3}.[0-9]{1,3}.[0-9]{1,3}.[0-9]{1,3}", score=0.8)

# anonymizer.add_recognizer(PatternRecognizer(supported_entity="SYSTEM_NAME", patterns=[SYSTEM_NAME_pattern]))
# anonymizer.add_recognizer(PatternRecognizer(supported_entity="IC01", patterns=[IC01_pattern]))
# anonymizer.add_recognizer(PatternRecognizer(supported_entity="sdwan_ip_admin", patterns=[IPV4_IP_pattern]))

# for column in columns_to_anonymize_deny_list:
#     anonymizer.add_recognizer(PatternRecognizer(supported_entity=f"{column}",
#                                                 deny_list=list(dataset[column].unique()),
#                                                 deny_list_score=0.5)) 


# def anonymize_text(text):
#     return anonymizer.anonymize(text)


# def anonymize_dataframe(df):
#     for column in df.columns:
#         if column in columns_to_anonymize:
#             df[column] = df[column].astype(str)
#             df[column] = df[column].apply(anonymizer.anonymize)

#     return df


# def deanonimize_text(text):
#     return anonymizer.deanonymize(text)


# def deanonymize_dataframe(df):
#     for column in df.columns:
#         if column in columns_to_anonymize:
#             df[column] = df[column].astype(str)
#             df[column] = df[column].apply(anonymizer.deanonymize)

#     return df

In [4]:
# import pickle
# import pandas as pd

# df=pd.read_excel("Power BI data-Security.xlsx",skiprows=2)
# df_anonymized=anonymize_dataframe(df)
# df_anonymized.head()


In [5]:
# actual_df_1=df_anonymized.copy()
# for column in df_anonymized.columns:
#     if column in columns_to_anonymize:
#         actual_df_1[column] = actual_df_1[column].astype(str)
#         actual_df_1[column] = actual_df_1[column].apply(anonymizer.deanonymize)
  
# actual_df_1

In [6]:
# anonymizer.save_deanonymizer_mapping("anonymization_object.json")

In [7]:
# df_anonymized.to_csv("anonymized_data.csv")

In [8]:
# from langchain_experimental.data_anonymizer import PresidioReversibleAnonymizer

# # Initialize the AnonymizerEngine
# new_anonymizer =  PresidioReversibleAnonymizer(
#     analyzed_fields=[], # choose inbuilt fields to be considered for anonimyzation
#     faker_seed=42,
# )

# columns_to_anonymize = ["customer name","IC01 Code","sysname","TMKey","sdwan_ip_admin"] # list all the column names to anonymize

# new_anonymizer.load_deanonymizer_mapping("anonymization_object.json")

# def deanonymize_dataframe(df):
#     for column in df.columns:
#         if column in columns_to_anonymize:
#             df[column] = df[column].astype(str)
#             df[column] = df[column].apply(anonymizer.deanonymize)

#     return df



In [9]:
# actual_df_2=df_anonymized.head(10)
# for column in df_anonymized.columns:
#     if column in columns_to_anonymize:
#         actual_df_2[column] = actual_df_2[column].astype(str)
#         actual_df_2[column] = actual_df_2[column].apply(new_anonymizer.deanonymize)

# actual_df_2

# Add table columns data to the schema desciption file

In [10]:
# import pandas as pd

# feature_columns=["type_service","Service","TM Manager","country","region","Sales Country","Sales Region"]

# df = pd.read_csv("anonymized_data.csv")

# with open("desc.txt", 'r') as f_source:
#     file_content = f_source.read()

# with open("InfoChest.txt", "w") as file:
#     file.write(file_content)
#     file.write("\n\nUnique Column Values in the dataframe:\n\n")

#     for column in df.columns:
#         if column not in feature_columns:
#             continue
#         file.write(f"Column name: {column}\nUnique Values:\n")
#         unique_values = df[column].unique()
#         file.writelines(f"{value}\n" for value in unique_values)
#         file.write("\n")

# Sample Queries

In [11]:
# df=pd.read_excel("Power BI data-Security.xlsx",skiprows=2)

### Query-01

In [12]:
# query="Give all the device from the sales country uae"
# query=new_anonymizer.anonymize(query)
# error=""
# counter=0
# while True:
#     try:
#         prompt=generate_response("Provide only the Python code wrapped inside the function generated_response() with no args to "+query+error)
#         print("The generated script is:-")
#         print(prompt)
#         exec(prompt)
#         RESPONSE=generated_response()
#         break
#     except Exception as e:
#         print(e)
#         error=f"""got error as {e} on your previous response {prompt}, give correct code"""
#         pass
#     counter+=1
#     if counter>=5:
#         RESPONSE="No related information found in the dataframe"
#         break

# print()
# print("*"*100)
# print("response from the 1st llm is:- ")
# print("*"*100)
# print(RESPONSE)

# # df_anonymized=anonymize_dataframe(RESPONSE)
# # print()
# # print("*"*100)
# # print("The anonymized df is:-")
# # print(df_anonymized)

# # anonymized_query=anonymize_text(query)
# # anonymized_RESPONSE2=generate_response_from_filtered_data(df_anonymized,anonymized_query)
# # RESPONSE2=deanonimize_text(anonymized_RESPONSE2)

# # print()
# # print("*"*100)
# # print("response from the 2nd llm is:- ")
# # print(RESPONSE2)

### Query-02

In [13]:
# query="Give all the sdwan that are vul"
# error=""
# counter=0
# while True:
#     try:
#         prompt=generate_response("Provide only the Python code wrapped inside the function generated_response() to "+query+error)
#         print(prompt)
#         exec(prompt)
#         RESPONSE=generated_response()
#         break
#     except Exception as e:
#         print(e)
#         error=f"""got error as {e} on your previous response {prompt}."""
#         pass
#     counter+=1
#     if counter>=5:
#         RESPONSE="No related information found in the dataframe"
#         break

# print()
# print("*"*100)
# print("resonse from the 1st llm is:- ")
# print(RESPONSE)

# # df_anonymized=anonymize_dataframe(RESPONSE)
# # print()
# # print("*"*100)
# # print("The anonymized df is:-")
# # print(df_anonymized)

# # anonymized_query=anonymize_text(query)
# # anonymized_RESPONSE2=generate_response_from_filtered_data(df_anonymized,anonymized_query)
# # RESPONSE2=deanonimize_text(anonymized_RESPONSE2)

# # print()
# # print("*"*100)
# # print("response from the 2nd llm is:- ")
# # print(RESPONSE2)

### Query-03

In [14]:
# query="Give all the devices with ic01 – {690680}."
# error=""
# counter=0
# while True:
#     try:
#         prompt=generate_response("Provide only the Python code wrapped inside the function generated_response() to "+query+error)
#         print(prompt)
#         exec(prompt)
#         RESPONSE=generated_response()
#         break
#     except Exception as e:
#         print(e)
#         error=f"""got error as {e} on your previous response {prompt}."""
#         pass
#     counter+=1
#     if counter>=5:
#         RESPONSE="No related information found in the dataframe"
#         break

# print()
# print("*"*100)
# print("resonse from the 1st llm is:- ")
# print(RESPONSE)

# # df_anonymized=anonymize_dataframe(RESPONSE)
# # print()
# # print("*"*20)
# # print("The anonymized df is:-")
# # print(df_anonymized)

# # anonymized_query=anonymize_text(query)
# # anonymized_RESPONSE2=generate_response_from_filtered_data(df_anonymized,anonymized_query)
# # RESPONSE2=deanonimize_text(anonymized_RESPONSE2)

# # print()
# # print("*"*20)
# # print("response from the 2nd llm is:- ")
# # print(RESPONSE2)

### LLM to restructure query to match the dataframe features

In [15]:
import pandas as pd
import numpy as np
import os
from openai import AzureOpenAI
from dotenv import load_dotenv

# del os.environ["API_VERSION_4o"]
load_dotenv()

# print(os.getenv("AZURE_OPENAI_4o_API_KEY"))
# print(os.getenv("API_VERSION_4o"))
# print(os.getenv("AZURE_OPENAI_4o_ENDPOINT"))

def generate_query(query):
    dfvalues=""
    with open("temp/InfoChest.txt","r") as file:
        for reader in file.readlines():
            dfvalues+=reader


    client = AzureOpenAI(
    api_key = os.getenv("AZURE_OPENAI_4o_API_KEY"),  
    api_version = os.getenv("API_VERSION_4o"),
    azure_endpoint = os.getenv("AZURE_OPENAI_4o_ENDPOINT")
    )


    response = client.chat.completions.create(
        model="gpt-4o", # model = "deployment_name".
        messages=[
            {"role": "system", "content": f"""You have a dataframe information as an input. \
        It contains column namea and the unique values of all the columns. \

    {dfvalues}

    Don't asssume any other column and feature name, it should be exactly the same. \
    Given a user query, replace the given column names and the column values with the actual ones from that column, make sure to not remove anything. \
    Do not change or replace values within <> . \
    Do not alter placeholder values . \
    Restructure the query for better understanding of columns and its values. \
    Just give the new query directly"""},

    {"role": "user", "content": f"{query}"},


        ]
    )

    # print(response.choices[0].message.content)

    # script=""
    # for line in response.choices[0].message.content.split("```")[1].split("\n")[1:]:
    #     script+=line+"\n"

    script=response.choices[0].message.content
    # temp=script.split("```")[1]
    # script=temp[6:]

    return script

# Testing

In [16]:
df=pd.read_parquet("processed_data/raw_data.parquet")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53997 entries, 0 to 53996
Data columns (total 56 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer name             53997 non-null  object
 1   IC01 Code                 53997 non-null  object
 2   sysname                   53997 non-null  object
 3   #clean device             53997 non-null  bool  
 4   #PSIRT Vln                53997 non-null  bool  
 5   #SDWAN vln                53997 non-null  bool  
 6   YOGI_COG issues           53997 non-null  bool  
 7   not SSHv2                 53997 non-null  bool  
 8   not SNMPv3                53997 non-null  bool  
 9   EOL                       53997 non-null  bool  
 10  Non Standard              53997 non-null  bool  
 11  country                   53997 non-null  object
 12  region                    53997 non-null  object
 13  chassis_api               53997 non-null  object
 14  ios_version           

In [17]:
# from langchain_experimental.data_anonymizer import PresidioReversibleAnonymizer

# # Initialize the AnonymizerEngine
# new_anonymizer =  PresidioReversibleAnonymizer(
#     analyzed_fields=[], # choose inbuilt fields to be considered for anonimyzation
#     faker_seed=42,
# )

# columns_to_anonymize = ["customer name","IC01 Code","sysname","TMKey","sdwan_ip_admin"]

# new_anonymizer.load_deanonymizer_mapping("anonymization_object.json")

# new_anonymizer.analyzed_fields=["EMAIL"]

# def anonymize_text(text):
#     return new_anonymizer.anonymize(text)


# def anonymize_dataframe(df):
#     for column in df.columns:
#         if column in columns_to_anonymize:
#             df[column] = df[column].astype(str)
#             df[column] = df[column].apply(new_anonymizer.anonymize)

#     return df


# def deanonymize_text(text):
#     return new_anonymizer.deanonymize(text)


# def deanonymize_dataframe(df):
#     for column in df.columns:
#         if column in columns_to_anonymize:
#             df[column] = df[column].astype(str)
#             df[column] = df[column].apply(new_anonymizer.deanonymize)

#     return df

In [18]:
from anonymizer import *

In [19]:
import tiktoken
enc = tiktoken.encoding_for_model("gpt-4o")
total_tokens=0
tokens_llm_1=[]
tokens_llm_2=[]
tokens_llm_3=[]

dfvalues=""
with open("temp/InfoChest.txt","r") as file:
    for reader in file.readlines():
        dfvalues+=reader

schema_info=""
with open("temp/data_description.txt","r") as file:
    for reader in file.readlines():
        schema_info+=reader

costpertokenin=0.005/1000
costpertokenout=0.015/1000

query="Give all the devices with ic01 – {60640}, only the system name and comapny"

print("*"*100)
print("Original Query")
print("*"*100)
print(query)
print("\n\n")

anonymized_query=anonymize_text(query)

print("*"*100)
print("Anonymized Query")
print("*"*100)
print(anonymized_query)
print("\n\n")

tokens_llm_1.append(len(enc.encode(anonymized_query+f"""You have a dataframe information as an input. \
        It contains column namea and the unique values of all the columns. \

    {dfvalues}

    Don't asssume any other column and feature name, it should be exactly the same. \
    Given a user query, replace the given column names and the column values with the actual ones from that column, make sure to not remove anything. \
    Do not change or replace values within <> . \
    Do not alter placeholder values . \
    Restructure the query for better understanding of columns and its values. \
    Just give the new query directly""")))
restructred_query_anonymized = generate_query(anonymized_query)
tokens_llm_1.append(len(enc.encode(restructred_query_anonymized)))

print("*"*100)
print("Restructured Anonymized Query")
print("*"*100)
print(restructred_query_anonymized)
print("\n\n")


print("*"*100)
print("Generated pandas Script (Anonymized)")
print("*"*100)

error=""
counter=0
while True:
    try:
        tokens_llm_2.append(len(enc.encode("Provide only the Python code wrapped inside the function generated_response() with no args to "+restructred_query_anonymized+error+" and generate a dataframe"+f"""You have access to a pandas dataframe named df. \
    Here is a name and decription of each column in the dataframe

    {schema_info}

    Don't asssume any other column and feature name, it should be exactly the same. \
    Given a user question about the dataframe,Provide only the Python code to answer it.\
    Don't assume you have access to any libraries other than built-in Python ones and pandas only. \
    You don't have any other data source other than dataframe df. \
    Make sure to refer only to the variables mentioned above. \
    Give the script to wrap it inside the custom function name generated_response(). \
    Return a single, new dataframe with only relevant and suitable column names according to the user query without modifying df \
    Give only required columns. \
    Do not add any print statements. \
    Do not alter placeholder values . \
    DO NOT REMOVE ANGULAR BRACKETS""")))
        anonymized_prompt=generate_response("Provide only the Python code wrapped inside the function generated_response() with no args to "+restructred_query_anonymized+error+" and generate a dataframe")
        tokens_llm_2.append(len(enc.encode(anonymized_prompt)))
        print(anonymized_prompt)
        print("\n\n")

        print("*"*100)
        print("Generated pandas Script (Deanonymized)")
        print("*"*100)

        prompt=deanonymize_text(anonymized_prompt)
        print(prompt)

        exec(prompt)
        RESPONSE=generated_response()
        # RESPONSE=deanonymize_dataframe(RESPONSE)
        break
    except Exception as e:
        print(e)
        error=f"""got error as {e} on your previous response {prompt}, give correct code"""
        pass
    counter+=1
    if counter>=5:
        RESPONSE="Could not generate response, try restructuring your query !"
        break

print("\n\n")
print("*"*100)
print("response from the 1st llm is:- ")
print("*"*100)
print(RESPONSE)


print("\n\n")
print("*"*100)
print("response from the 2nd llm is:- ")
print("*"*100)

if RESPONSE.shape[0]*RESPONSE.shape[1] > 5000:
    print("Cannot be processsed further due to large size")

else:
    tokens_llm_3.append(len(enc.encode(f"""
             You are responsible for explaining the answer to the user based on the dataframe, schema and the query provided to you.\
    The following dataframe is already filtered according to user query and columns renamed: {anonymize_dataframe(RESPONSE).to_csv(index=False)} \
    Assume it contains all the information asked in query. \
    This is the description of original dataset: {schema_info} \
    If query cannot be answered from the provided data, simply explain the reason for the same.\
    Do not give any tabular data. \
    Do not include any reasonings and warnings, directly respond to the query. \
    Do not give any warnings. \
    Do not remove angular <> brackets, keep those values as it is. \
    Do not alter placeholder values . \
    Empty dataframe means there were no such records.""")))
    filtered_RESPONSE=deanonymize_text(generate_response_from_filtered_data(dataframe=anonymize_dataframe(RESPONSE),query=anonymized_query))
    tokens_llm_3.append(len(enc.encode(filtered_RESPONSE)))
    print(filtered_RESPONSE)


print("\n\n")
print("*"*100)
print("Respective costs for llms are:- ")
print("*"*100)
print("tokens used by the query_restructuring llm:",tokens_llm_1,"cost:",tokens_llm_1[0]*costpertokenin+tokens_llm_1[1]*costpertokenout)
print("tokens used by the filtration llm:",tokens_llm_2,"cost:",tokens_llm_2[0]*costpertokenin+tokens_llm_2[1]*costpertokenout)
print("tokens used by response llm:",tokens_llm_3,"cost:",tokens_llm_3[0]*costpertokenin+tokens_llm_3[1]*costpertokenout)
print("total cost($) for the query:", tokens_llm_1[0]*costpertokenin+tokens_llm_1[1]*costpertokenout+tokens_llm_2[0]*costpertokenin+tokens_llm_2[1]*costpertokenout+tokens_llm_3[0]*costpertokenin+tokens_llm_3[1]*costpertokenout)


****************************************************************************************************
Original Query
****************************************************************************************************
Give all the devices with ic01 – {60640}, only the system name and comapny



****************************************************************************************************
Anonymized Query
****************************************************************************************************
Give all the devices with ic01 – {<IC01 Code placeholder>}, only the system name and comapny



****************************************************************************************************
Restructured Anonymized Query
****************************************************************************************************
Give all the devices with IC01 Code – {<IC01 Code placeholder>}, only the sysname and customer name



*******************************************************